In [119]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf

import warnings
warnings.filterwarnings('ignore')

In [120]:
datapath = '/Users/yufanwang/Desktop/MarmosetWM_Project/revision_NC/af_projection/result'

In [ ]:
human_af_l_all = np.loadtxt(f'{datapath}/probtrackx2/human_af_l_thr0_norm.txt')
chimp_af_l_all = np.loadtxt(f'{datapath}/probtrackx2/chimp_af_l_thr0_norm.txt')
macaque_af_l_all = np.loadtxt(f'{datapath}/probtrackx2/macaque_tvb_af_l_thr0_norm.txt')
marmoset_af_l_all = np.loadtxt(f'{datapath}/probtrackx2/marmoset_MBM_af_l_thr0_norm.txt')
macaque_ucd_af_l_all = np.loadtxt(f'{datapath}/ucd_minds/macaque_ucd_af_l_thr0_norm.txt')
marmoset_minds_af_l_all = np.loadtxt(f'{datapath}/ucd_minds/marmoset_minds_af_l_thr0_norm.txt')

human_af_r_all = np.loadtxt(f'{datapath}/probtrackx2/human_af_r_thr0_norm.txt')
chimp_af_r_all = np.loadtxt(f'{datapath}/probtrackx2/chimp_af_r_thr0_norm.txt')
macaque_af_r_all = np.loadtxt(f'{datapath}/probtrackx2/macaque_tvb_af_r_thr0_norm.txt')
marmoset_af_r_all = np.loadtxt(f'{datapath}/probtrackx2/marmoset_MBM_af_r_thr0_norm.txt')
macaque_ucd_af_r_all = np.loadtxt(f'{datapath}/ucd_minds/macaque_ucd_af_r_thr0_norm.txt')
marmoset_minds_af_r_all = np.loadtxt(f'{datapath}/ucd_minds/marmoset_minds_af_r_thr0_norm.txt')

In [ ]:
index_human_chimp_L = np.loadtxt(f'/Users/yufanwang/Desktop/MarmosetWM_Project/revision_NC/indi_bp_atlas/index_human_chimp_L.txt')
index_human_macaque_L = np.loadtxt(f'/Users/yufanwang/Desktop/MarmosetWM_Project/revision_NC/indi_bp_atlas/index_human_macaque_L.txt')
index_human_marmoset_L = np.loadtxt(f'/Users/yufanwang/Desktop/MarmosetWM_Project/revision_NC/indi_bp_atlas/index_human_marmoset_L.txt')

## construct data

In [135]:
def read_subject_list(txt_path):
    with open(txt_path, "r") as f:
        subs = [line.strip() for line in f if line.strip()]
    return subs

def build_species_df_site(list_path, species_name, site, left_vals, right_vals, region):
    subj = read_subject_list(list_path)
    L = np.asarray(left_vals).reshape(-1)
    R = np.asarray(right_vals).reshape(-1)

    if not (len(subj) == len(L) == len(R)):
        raise ValueError(
            f"[{species_name}] length mismatch: "
            f"subjects={len(subj)}, L={len(L)}, R={len(R)}"
        )

    df_L = pd.DataFrame({
        "subject_id": subj,
        "species": species_name,
        "site": site,
        "hemisphere": "LH",
        "region": region,
        "connectivity_score": L
    })

    df_R = pd.DataFrame({
        "subject_id": subj,
        "species": species_name,
        "site": site,
        "hemisphere": "RH",
        "region": region,
        "connectivity_score": R
    })

    return pd.concat([df_L, df_R], ignore_index=True)

In [136]:
human_list_path = "/Users/yufanwang/Desktop/MarmosetWM_Project/revision_NC/indi_bp_atlas/humanlist40.txt"
chimp_list_path = "/Users/yufanwang/Desktop/MarmosetWM_Project/revision_NC/indi_bp_atlas/chimplist46.txt"
macaque_list_path = "/Users/yufanwang/Desktop/MarmosetWM_Project/revision_NC/indi_bp_atlas/macaque_tvb_list.txt"
marmoset_list_path = "/Users/yufanwang/Desktop/MarmosetWM_Project/revision_NC/indi_bp_atlas/marmoset_MBM_list.txt"
macaque_ucd_list_path = "/Users/yufanwang/Desktop/MarmosetWM_Project/revision_NC/indi_bp_atlas/macaque_ucdavis_list.txt"
marmoset_minds_list_path = "/Users/yufanwang/Desktop/MarmosetWM_Project/revision_NC/indi_bp_atlas/marmoset_minds_list.txt"

i = 17
region = "A45c"

roi_index_human = i - 1
roi_index_chimp = index_human_chimp_L[roi_index_human] - 1
roi_index_macaque = index_human_macaque_L[roi_index_human] - 1
roi_index_marmoset = index_human_marmoset_L[roi_index_human] - 1

value_human_L = human_af_l_all[:,roi_index_human]
value_chimp_L = chimp_af_l_all[:,int(roi_index_chimp)]
value_macaque_L = macaque_af_l_all[:,int(roi_index_macaque)]
value_marmoset_L = marmoset_af_l_all[:,int(roi_index_marmoset)]
value_macaque_ucd_L = macaque_ucd_af_l_all[:,int(roi_index_macaque)]
value_marmoset_minds_L = marmoset_minds_af_l_all[:,int(roi_index_marmoset)]

value_human_R = human_af_r_all[:,roi_index_human]
value_chimp_R = chimp_af_r_all[:,int(roi_index_chimp)]
value_macaque_R = macaque_af_r_all[:,int(roi_index_macaque)]
value_marmoset_R = marmoset_af_r_all[:,int(roi_index_marmoset)]
value_macaque_ucd_R = macaque_ucd_af_r_all[:,int(roi_index_macaque)]
value_marmoset_minds_R = marmoset_minds_af_r_all[:,int(roi_index_marmoset)]

df = pd.concat([
    build_species_df_site(human_list_path,    "human",      "HCP", value_human_L,    value_human_R, region),
    build_species_df_site(chimp_list_path,    "chimpanzee", "NCBR", value_chimp_L,    value_chimp_R, region),
    build_species_df_site(macaque_ucd_list_path,  "macaque", "ucd",    value_macaque_ucd_L,  value_macaque_ucd_R, region),
    build_species_df_site(marmoset_minds_list_path, "marmoset", "minds",   value_marmoset_minds_L, value_marmoset_minds_R, region),
], ignore_index=True)

print(df.head())
print(df.shape)  # 应该是 2 * (40+46+8+24) = 236 行

df.to_csv(f'{datapath}/af_projection_{region}_ucd_minds.csv', index=False)

## macaque only
df_macaque = pd.concat([
    build_species_df_site(macaque_list_path,  "macaque", "tvb",    value_macaque_L,  value_macaque_R, region),
    build_species_df_site(macaque_ucd_list_path,  "macaque", "ucd",    value_macaque_ucd_L,  value_macaque_ucd_R, region),
], ignore_index=True)

print(df_macaque.head())
print(df_macaque.shape)  # 应该是 2 * (40+46+8+24) = 236 行

df_macaque.to_csv(f'{datapath}/af_projection_{region}_ucd_minds_macaque.csv', index=False)

## marmoset only
df_marmoset = pd.concat([
    build_species_df_site(marmoset_list_path, "marmoset", "MBM",   value_marmoset_L, value_marmoset_R, region),
    build_species_df_site(marmoset_minds_list_path, "marmoset", "minds",   value_marmoset_minds_L, value_marmoset_minds_R, region),
], ignore_index=True)

print(df_marmoset.head())
print(df_marmoset.shape)  # 应该是 2 * (40+46+8+24) = 236 行

df_marmoset.to_csv(f'{datapath}/af_projection_{region}_ucd_minds_marmoset.csv', index=False)

  subject_id species site hemisphere region  connectivity_score
0     100307   human  HCP         LH   A45c            0.335231
1     100408   human  HCP         LH   A45c            0.339382
2     103414   human  HCP         LH   A45c            0.133474
3     105115   human  HCP         LH   A45c            0.264675
4     106016   human  HCP         LH   A45c            0.541276
(430, 6)
  subject_id  species site hemisphere region  connectivity_score
0     sub-01  macaque  tvb         LH   A45c            0.013393
1     sub-02  macaque  tvb         LH   A45c            0.097818
2     sub-03  macaque  tvb         LH   A45c            0.006349
3     sub-04  macaque  tvb         LH   A45c            0.000000
4     sub-05  macaque  tvb         LH   A45c            0.000000
(54, 6)
   subject_id   species site hemisphere region  connectivity_score
0  sub-NIHm14  marmoset  MBM         LH   A45c            0.180736
1  sub-NIHm15  marmoset  MBM         LH   A45c            0.050014
2  sub-N